# 10RE - Práctica de Métricas en Aprendizaje No Supervisado

En este notebook pondrás en práctica las métricas más importantes para evaluar el clustering, usando los datasets **abalone.csv** y **student_exam_score.csv**.

## ¿Qué aprenderás aquí?
- Cómo aplicar el método del codo y entender la inercia.
- Cómo interpretar el coeficiente de silueta.
- Qué significan las métricas Davies-Bouldin y Calinski-Harabasz.
- Cómo comparar y justificar el número óptimo de clústeres.

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

## 1. Cargar y preparar los datos
Trabajaremos con los archivos abalone.csv y student_exam_score.csv.

In [ ]:
# Cargar datasets
abalone = pd.read_csv('abalone.csv')
student = pd.read_csv('student_exam_scores.csv')

# Seleccionar solo variables numéricas
abalone_num = abalone.select_dtypes(include=[np.number])
student_num = student.select_dtypes(include=[np.number])

# Escalar
scaler_abalone = StandardScaler()
X_abalone = scaler_abalone.fit_transform(abalone_num)

scaler_student = StandardScaler()
X_student = scaler_student.fit_transform(student_num)

## 2. Método del Codo (Elbow)
El método del codo nos ayuda a elegir el número de clústeres adecuado. La inercia mide lo juntos que están los puntos en cada grupo. Buscamos el 'codo' en la gráfica, donde la inercia deja de bajar rápidamente.

In [ ]:
def plot_elbow(X, title):
    inertias = []
    for k in range(1, 11):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(X)
        inertias.append(kmeans.inertia_)
    plt.plot(range(1, 11), inertias, marker='o')
    plt.xlabel('Número de clústeres')
    plt.ylabel('Inercia')
    plt.title(f'Método del codo {title}')
    plt.show()

plot_elbow(X_abalone, 'Abalone')
plot_elbow(X_student, 'Student Exam Score')

**¿Qué debes observar?**
Busca el punto donde la curva de inercia cambia de pendiente bruscamente. Ese es el número óptimo de clústeres.

## 3. Silhouette Score
El coeficiente de silueta mide lo bien agrupado que está cada punto. Si es cercano a 1, el punto está bien en su grupo. Si es cercano a 0, está entre dos grupos. Si es negativo, está mal clasificado.

In [ ]:
def plot_silhouette_scores(X, title):
    scores = []
    for k in range(2, 11):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X)
        score = silhouette_score(X, labels)
        scores.append(score)
    plt.plot(range(2, 11), scores, marker='o')
    plt.xlabel('Número de clústeres')
    plt.ylabel('Silhouette Score')
    plt.title(f'Silhouette Score {title}')
    plt.show()

plot_silhouette_scores(X_abalone, 'Abalone')
plot_silhouette_scores(X_student, 'Student Exam Score')

**¿Qué debes observar?**
Elige el número de clústeres donde el Silhouette Score sea más alto.

## 4. Davies-Bouldin y Calinski-Harabasz
Estas métricas también ayudan a comparar la calidad de los clústeres. Davies-Bouldin debe ser lo más bajo posible. Calinski-Harabasz, lo más alto posible.

In [ ]:
def plot_db_ch(X, title):
    db_scores = []
    ch_scores = []
    for k in range(2, 11):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(X)
        db_scores.append(davies_bouldin_score(X, labels))
        ch_scores.append(calinski_harabasz_score(X, labels))
    fig, ax1 = plt.subplots()
    ax1.plot(range(2, 11), db_scores, 'r-o', label='Davies-Bouldin (bajo=mejor)')
    ax1.set_xlabel('Número de clústeres')
    ax1.set_ylabel('Davies-Bouldin', color='r')
    ax2 = ax1.twinx()
    ax2.plot(range(2, 11), ch_scores, 'b-o', label='Calinski-Harabasz (alto=mejor)')
    ax2.set_ylabel('Calinski-Harabasz', color='b')
    plt.title(f'Davies-Bouldin y Calinski-Harabasz {title}')
    plt.show()

plot_db_ch(X_abalone, 'Abalone')
plot_db_ch(X_student, 'Student Exam Score')

## 5. Práctica final
Elige el mejor número de clústeres para cada dataset y justifica tu elección usando las métricas anteriores.

- ¿Coinciden todas las métricas en el mismo número de clústeres?
- ¿Qué diferencias observas entre los dos datasets?

**Reflexión sobre los resultados en Student Exam Score:**

En este caso, la mayoría de las métricas (Silhouette Score y Davies-Bouldin) indican que el número óptimo de clústeres es **2**. Sin embargo, la métrica Calinski-Harabasz muestra su valor máximo en **9** clústeres.

Esto puede ocurrir porque Calinski-Harabasz a veces favorece valores altos de clústeres, especialmente si los datos tienen pocos puntos o los grupos no son muy compactos. Por el contrario, Silhouette y Davies-Bouldin suelen ser más fiables porque consideran tanto la cohesión interna como la separación entre grupos.

**Conclusión:**
Me quedo con **2 clústeres** como la mejor opción para este dataset, ya que está respaldado por la mayoría de las métricas y tiene más sentido práctico. Es importante comparar varias métricas y no basarse solo en una para tomar la decisión final.